In [1]:
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
with open("../data/train.fasta", "r") as file:
    train_fasta = [line.rstrip('\n') for line in file]

with open("../data/blind.fasta", "r") as file:
    blind_fasta = [line.rstrip('\n') for line in file]

with open("../data/train.dssp", "r") as file:
    train_dssp = [line.rstrip('\n') for line in file]

with open("../data/blind.dssp", "r") as file:
    blind_dssp = [line.rstrip('\n') for line in file]


maxlen_seq = 760

In [12]:
train_fasta

['KKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENVMERKKPALVSIPEDVVQAEAGGMHTVCLSKSGQVYSFGCNDEGALGRDTSVEGSEMVPGKVELQEKVVQVSAGDSHTAALTDDGRVFLWGSFRDNNGVIGLLEPMKKSMVPVQVQLDVPVVKVASGNDHLVMLTADGDLYTLGCGEQGQLGRVPELFANRGGRQGLERLLVPKCVMLKSRGSRGHVRFQDAFCGAYFTFAISHEGHVYGFGLSNYHQLGTPGTESCFIPQNLTSFKNSTKSWVGFSGGQHHTVCMDSEGKAYSLGRAEYGRLGLGEGAEEKSIPTLISRLPAVSSVACGASVGYAVTKDGRVFAWGMGTNYQLGTGQDEDAWSPVEMMGKQLENRVVLSVSSGGQHTVLLVKDKEQS',
 'AGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQIQVPLGDAARPSHLLTSQLPLMWQLYPEERYMDNNSRLWQIQHHLMVRGVQELLLKLLPDD',
 'MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSGE',
 'EVGATGFDPKVSLDDPEALTKIRRELKDAGADRIWYIADAFRAGLSVDGVFNLTNIDRWFLVQIEELVRLEEKVAEVGITGLNADFLRQLKRKGFADARLAKLAGVREAEIRKLRDQYDLHPVYKRVDTCAAEFATDTAYMYSTYEEECEANP',
 'VDVSPTSQRLQLLEPFDKWDGKDLEDLQILIKVKGKCTTDHISAAGPWLKFRGHLDNISNNLLIGAINSENRKANSVRNAVTQEFGPVPDTARYYKQHGIRWVVIGDENYGEGSSREHSALEPRFLGGRAIITKSFARIHETNLKKQGLLPLTFADPADYNKIHPVDKLTIQGLKDFAPGKPLTCIIKHPNGTQETILLNHTFNETQIEWFRAGSALNRMKELQQK',
 'RAKVAMSHFEPHEYIRYDLLEKNIDIVRKRLNRPLTLSEK

In [4]:
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example


def data_transform(lines):
    list_fin = []
    for i in range(len(lines)):
        data = list(lines[i])
        data.append(" ")
        while(len(data) != 760):
            data.append(" ")
        values = array(data)

        # integer encode
        label_encoder = LabelEncoder()
        integer_encoded = label_encoder.fit_transform(values)

        data = integer_encoded
        data = array(data)
        # one hot encode
        encoded = to_categorical(data, num_classes=21)
        list_fin.append(encoded)
    return array(list_fin)

def data_transform_ddsp(lines):
    list_fin = []
    for i in range(len(lines)):
        data = list(lines[i])
        data.append(" ")
        while(len(data) != 760):
            data.append(" ")
        values = array(data)

        # integer encode
        label_encoder = LabelEncoder()
        integer_encoded = label_encoder.fit_transform(values)

        data = integer_encoded
        data = array(data)
        # one hot encode
        encoded = to_categorical(data, num_classes=4)
        list_fin.append(encoded)
    return array(list_fin)

X_train = data_transform(train_fasta)
X_test = data_transform(blind_fasta)
Y_train = data_transform_ddsp(train_dssp)
Y_test = data_transform_ddsp(blind_dssp)


In [5]:

X_train.shape, Y_train.shape

((1348, 760, 21), (1348, 760, 4))

In [6]:
X_test.shape, Y_test.shape

((149, 760, 21), (149, 760, 4))

In [7]:
import keras
import numpy as np

from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense, Flatten
from keras import Input, Model, Sequential
from keras.layers import add, Activation, TimeDistributed
#from keras.utils import plot_model  # Needs pydot.
from keras.layers import Conv1D, AveragePooling1D, Conv2D,AveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [8]:
def my_model7():
    n_residual = 4
    print("Simple residual network with {} modules".format(n_residual))
    inputs = Input(shape=(760, 21))
    n_filters=21
    conv_1 = Conv1D(n_filters, (3), padding="same", activation="relu",
                    kernel_initializer="he_normal")(inputs)
    conv_2 = Conv1D(n_filters, (3), padding="same", activation="linear",
                    kernel_initializer="he_normal")(conv_1)    
    output = TimeDistributed(Dense(4, activation="softmax"))(conv_2)

    model = Model(inputs=inputs, outputs=output)

    print(model.summary())

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model


In [9]:
model = my_model7()
# Training the model;
model.fit(X_train, Y_train, batch_size=20, epochs=20, validation_data = (X_test,Y_test))
# model.fit(X_train, Y_train, batch_size=20, epochs=20)


# Perf of the model.
model.evaluate(X_train, Y_train)


Simple residual network with 4 modules
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 760, 21)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 760, 21)           1344      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 760, 21)           1344      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 760, 4)            88        
Total params: 2,776
Trainable params: 2,776
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 1348 samples, validate on 149 samples
Epoch 1/20
1348/1348 [==============================] - 9s 6ms/step -

[0.208124435443199, 0.8993254899978638]

In [10]:
predictions = model.predict(X_test)

tp=0
fp=0
tn=0
fn=0

tot=0

for i in range(len(predictions)):
    for j in range(len(predictions[i])):
        if Y_test[i,j,3] != 0 :
            predmax=-1
            predict_class= -1
            true_class = -1
            for k in range(len(predictions[i,j])):
                if predmax < predictions[i,j,k]:
                    predmax = predictions[i,j,k]
                    predict_class = k
                if Y_test[i,j,k] == 1:
                    true_class=k
            if predict_class == true_class:
                tp=tp+1
            tot=tot+1

print(tp)
print(tot)
print("ACC  %f"%((tp/tot)*100))

2196
5257
ACC  41.772874


In [7]:
def residual_module(lay_i, n_filters):
    if lay_i.shape[-1] != n_filters:
        lay_i = Conv1D(n_filters, (1), padding="same", activation="relu",kernel_initializer="he_normal")(lay_i)
    save = lay_i
    conv_1 = Conv1D(n_filters, (3), padding="same", activation="relu",kernel_initializer="he_normal")(lay_i)
    conv_2 = Conv1D(n_filters, (3), padding="same", activation="linear",kernel_initializer="he_normal")(conv_1)
    conc_1 = add([conv_2, save])
    output = Activation("relu")(conc_1)

    return output

def my_model7():
    n_residual = 5
    print("Simple residual network with {} modules".format(n_residual))
    inputs = Input(shape=(760, 21))
    residual_i = inputs
    for _ in range(n_residual):
        residual_i = residual_module(residual_i, 24)

    output = TimeDistributed(Dense(4, activation="softmax"))(residual_i)


    model = Model(inputs=inputs, outputs=output)

    print(model.summary())

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

from tensorflow.keras.callbacks import TensorBoard
Name = "ResNet"
tensorboard = TensorBoard(log_dir="logs/{}".format(Name))

best_model_file = './my_model7'+'.h5'
best_model = ModelCheckpoint(best_model_file, monitor ='val_loss',verbose=2,save_best_only=True)

early_stopping = EarlyStopping(monitor='val_loss', patience=7,verbose=1,mode='min')

# Creating the model.
model = my_model7()
# Training the model;
model.fit(X_train, Y_train, batch_size=30, epochs=200, callbacks= [tensorboard,best_model,early_stopping]
          , validation_data=(X_test,Y_test))

Simple residual network with 5 modules




__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 760, 21)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 760, 24)      528         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 760, 24)      1752        conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 760, 24)      1752        conv1d_2[0][0]                   
__________________________________________________________________

In [8]:
predictions = model.predict(X_test)

tp=0
fp=0
tn=0
fn=0

tot=0

for i in range(len(predictions)):
    for j in range(len(predictions[i])):
        if Y_test[i,j,3] != 0 :
            predmax=-1
            predict_class= -1
            true_class = -1
            for k in range(len(predictions[i,j])):
                if predmax < predictions[i,j,k]:
                    predmax = predictions[i,j,k]
                    predict_class = k
                if Y_test[i,j,k] == 1:
                    true_class=k
            if predict_class == true_class:
                tp=tp+1
            tot=tot+1

print(tp)
print(tot)
print("ACC  %f"%((tp/tot)*100))

3429
5257
ACC  65.227316
